### MSc IE Thesis: Dual-Lever, Scenario-Based Hybrid MFA-LCA Framework

Author: Che (Chester) Xiao

The code includes three key aspects:\
    - Dual-lever scenario matrix:    three SSP2×RCPs (6.0, 4.5, 2.6) + three recycling regimes (BAU, Moderate, Aggressive) \
    - Dynamic MFA integration:       cohort-based stock, inflow-outflow via Weibull survival \
    - Modular prospective LCA (CFF): production, operation and EoL module with P[1]-P[7] processes from `carculator_bus` and TNO model, interpolated across 2026-2050   

In [1]:
from pathlib import Path
import re
import logging
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import textwrap
from functions_AEF_PEF import load_lca_data, interpolate_to_annual, compute_footprints_all, plot_interpolated, compute_footprints_with_components

# Configure logging
logging.basicConfig(level=logging.INFO)

# Directories
BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR / 'data'
RESULTS_DIR = BASE_DIR / 'results'
RESULTS_DIR.mkdir(exist_ok=True)

In [2]:
# Read allocation factor
A = 0.2                                           # Allocation factor
A_int = int(round(A*10))                          # 0.2 → 2
A_str = f"{A_int:02d}"                            # → "02"


In [3]:
# ___ 1. Load & Interpolate all LCA results ___

# Define LCA files
lca_files = {
    'RCP26': 'TIAM-UCL-SSP2-RCP26.csv',
    'RCP45': 'TIAM-UCL-SSP2-RCP45.csv',
    'RCP60': 'TIAM-UCL-SSP2-RCP60.csv'
}

# Batch process
dfs = []
for rcp, fname in lca_files.items():
    raw = load_lca_data(DATA_DIR / fname)
    interp = interpolate_to_annual(raw)
    interp['RCP'] = rcp
    dfs.append(interp)

# Combine & save
df_lca_all = pd.concat(dfs, ignore_index=True)
df_lca_all.to_csv(RESULTS_DIR / 'interpolated_all_unit_processes.csv', index=False)
logging.info('Saved interpolated_all_unit_processes.csv')

2025-05-12 09:58:06,550 - INFO - Loaded TIAM-UCL-SSP2-RCP26.csv: methods=Index(['EF v3.1, acidification, accumulated exceedance (AE)',
       'EF v3.1, climate change, global warming potential (GWP100)',
       'EF v3.1, climate change: biogenic, global warming potential (GWP100)',
       'EF v3.1, climate change: fossil, global warming potential (GWP100)',
       'EF v3.1, climate change: land use and land use change, global warming potential (GWP100)',
       'EF v3.1, ecotoxicity: freshwater, comparative toxic unit for ecosystems (CTUe)',
       'EF v3.1, ecotoxicity: freshwater, inorganics, comparative toxic unit for ecosystems (CTUe)',
       'EF v3.1, ecotoxicity: freshwater, organics, comparative toxic unit for ecosystems (CTUe)',
       'EF v3.1, energy resources: non-renewable, abiotic depletion potential (ADP): fossil fuels',
       'EF v3.1, eutrophication: freshwater, fraction of nutrients reaching freshwater end compartment (P)',
       'EF v3.1, eutrophication: marine, fr

In [4]:
# ___ 2. Interpolate data ___
methods_list = df_lca_all['method'].unique().tolist()
rcp_list     = list(lca_files.keys())
# plot_interpolated(df_lca_all, methods_list[:19], rcp_list)

In [5]:
# ___ 3. Compute footprints Across LCA results

# Load MFA outputs
df_mfa = pd.read_csv(DATA_DIR / 'MFA-outputs.csv')

# Compute AEF/CEF
results_all = compute_footprints_all(
    df_lca_all,
    df_mfa,
    methods_list,
    rcp_list,
    A=A,
    km_per_vehicle=50000,
    occupancy=10
)
# Save
out_csv = RESULTS_DIR / f"AEF_CEF_all_A_{A_str}.csv"
results_all.to_csv(out_csv, index=False)
logging.info(f"Saved {out_csv.name}")

2025-05-12 09:58:06,968 - INFO - Process → ID (forced ordering):
2025-05-12 09:58:06,968 - INFO -   1: 'passenger bus | P1_e_bus_production | CH\n| LFP_e_bus'
2025-05-12 09:58:06,968 - INFO -   2: 'transport, passenger bus |\nP2_e_bus_operation | CH | LFP_e_bus'
2025-05-12 09:58:06,968 - INFO -   3: 'used bus | P3_e_bus_eol_without_battery\n| CH | LFP_e_bus'
2025-05-12 09:58:06,968 - INFO -   4: 'used Li-ion battery | P4_LFP_eol | GLO |\nLFP_e_bus'
2025-05-12 09:58:06,968 - INFO -   5: 'lithium carbonate |\nP5_Li_virgin_production | RoW |\nLFP_e_bus'
2025-05-12 09:58:06,968 - INFO -   6: 'used LFP battery | P6_LFP_recycling |\nGLO | LFP_e_bus'
2025-05-12 09:58:06,968 - INFO -   7: 'lithium carbonate | P7_Li_recovery | GLO\n| LFP_e_bus'
2025-05-12 09:58:07,687 - INFO - Saved AEF_CEF_all_A_02.csv


In [6]:
# ___ 3.1 Compute footprints Across LCA results with detailed component

# Load MFA outputs
df_mfa = pd.read_csv(DATA_DIR / 'MFA-outputs.csv')

# Compute detailed footprints:
detailed_results = compute_footprints_with_components(
    df_lca_all,
    df_mfa,
    methods_list,
    rcp_list,
    A=A,
    km_per_vehicle=50000,
    occupancy=10
)

# Save the new CSV
out_detailed_csv = RESULTS_DIR / f"AEF_CEF_components_A_{A_str}.csv"
detailed_results.to_csv(out_detailed_csv, index=False)
logging.info(f"Saved {out_detailed_csv.name}")


2025-05-12 09:58:07,711 - INFO - Process → ID (forced ordering):
2025-05-12 09:58:07,711 - INFO -   1: 'passenger bus | P1_e_bus_production | CH\n| LFP_e_bus'
2025-05-12 09:58:07,711 - INFO -   2: 'transport, passenger bus |\nP2_e_bus_operation | CH | LFP_e_bus'
2025-05-12 09:58:07,711 - INFO -   3: 'used bus | P3_e_bus_eol_without_battery\n| CH | LFP_e_bus'
2025-05-12 09:58:07,711 - INFO -   4: 'used Li-ion battery | P4_LFP_eol | GLO |\nLFP_e_bus'
2025-05-12 09:58:07,715 - INFO -   5: 'lithium carbonate |\nP5_Li_virgin_production | RoW |\nLFP_e_bus'
2025-05-12 09:58:07,715 - INFO -   6: 'used LFP battery | P6_LFP_recycling |\nGLO | LFP_e_bus'
2025-05-12 09:58:07,716 - INFO -   7: 'lithium carbonate | P7_Li_recovery | GLO\n| LFP_e_bus'
2025-05-12 09:58:08,456 - INFO - Saved AEF_CEF_components_A_02.csv


In [7]:
# ___ 4. Plot AEF & CEF ___

# ensure plot_dir exists
plot_dir = RESULTS_DIR / f"plot_AEF_CEF_A_{A_str}"
plot_dir.mkdir(exist_ok=True)

# helper to make a Windows-safe filename
def sanitize(s: str) -> str:
    # 1) replace invalid chars with underscore
    safe = re.sub(r'[^\w\-]', '_', s)
    # 2) collapse runs of underscores
    safe = re.sub(r'_+', '_', safe)
    # 3) strip leading/trailing underscores
    return safe.strip('_')

# Styles for BAU, Moderate, Aggressive
styles = {
    'BAU':      {'linestyle':'-',  'marker':'o', 'markevery':5},
    'Moderate': {'linestyle':'--', 'marker':'s', 'markevery':5},
    'Aggressive':{'linestyle':':', 'marker':'^', 'markevery':5},
}

for method in methods_list:
    fig, axes = plt.subplots(
        nrows=len(rcp_list), ncols=2,
        figsize=(12, 4*len(rcp_list)),
        sharex=True
    )
    # Increase base font size
    plt.rcParams.update({'font.size': 11})

    for i, rcp in enumerate(rcp_list):
        df_m = results_all[
            (results_all['method'] == method) &
            (results_all['RCP']    == rcp)
        ]

        # plot each scenario with its style
        for sce, opts in styles.items():
            df_s = df_m[df_m['scenario'] == sce]
            axes[i,0].plot(
                df_s['year'], df_s['AEF'],
                label=sce,
                linewidth=2,
                **opts
            )
            axes[i,1].plot(
                df_s['year'], df_s['CEF'],
                label=sce,
                linewidth=2,
                **opts
            )

        # Titles
        axes[i,0].set_title(f"{rcp} — Annual Footprint (AEF)", pad=10)
        axes[i,1].set_title(f"{rcp} — Cumulative Footprint (CEF)", pad=10)

        # Style axes
        for ax in axes[i]:
            # only horizontal grid
            ax.yaxis.grid(True, linestyle='--', alpha=0.4)
            ax.xaxis.grid(False)
            ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=6, integer=True))
            plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
            # clean spines
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

    # shared y-labels
    axes[0,0].set_ylabel('AEF')
    axes[0,1].set_ylabel('CEF')

    fig.suptitle(f"Method: {method} A = {A:.2f}", fontsize=16, fontweight='bold', y=0.98)

    # shared legend at bottom
    handles, labels = axes[0,0].get_legend_handles_labels()
    fig.legend(
        handles, labels,
        title='Scenario',
        loc='lower center',
        ncol=3,
        bbox_to_anchor=(0.5, -0.02),
        fontsize=11,
        title_fontsize=12
    )

    # make room for legend
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    # save & close
    safe_method = sanitize(method)
    out_path = plot_dir / f"{safe_method}_AEF_CEF_A_{A_str}.png"
    fig.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

    print(f"Saved plot for {method} → {out_path}")
print('All plolt saved.')

Saved plot for EF v3.1, acidification, accumulated exceedance (AE) → E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\plot_AEF_CEF_A_02\EF_v3_1_acidification_accumulated_exceedance_AE_AEF_CEF_A_02.png
Saved plot for EF v3.1, climate change, global warming potential (GWP100) → E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\plot_AEF_CEF_A_02\EF_v3_1_climate_change_global_warming_potential_GWP100_AEF_CEF_A_02.png
Saved plot for EF v3.1, climate change: biogenic, global warming potential (GWP100) → E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\plot_AEF_CEF_A_02\EF_v3_1_climate_change_biogenic_global_warming_potential_GWP100_AEF_CEF_A_02.png
Saved plot for EF v3.1, climate change: fossil, global warming potential (GWP100) → E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\plot_AEF_CEF_A_02\EF_v3_1_climate_change_fossil_global_warming_potential_GWP100_AEF_CEF_A_02.png
Saved plot for EF v3.1, climate change: land use and land use change, global warming potential (GWP100) → E:\MSc_LCA\C_XIAO_MSc_IE_Thesis_2\results\